In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

In [ ]:
from pathlib import Path
import sys

# 1. "jpy-datareader" を見つけるまで上へたどる
project_root = Path.cwd()
while project_root.name != "jpy-datareader" and project_root != project_root.parent:
    project_root = project_root.parent

if project_root.name != "jpy-datareader":
    raise RuntimeError("プロジェクトルート 'jpy-datareader' が見つかりませんでした")

# 2. そのディレクトリを sys.path に追加
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# 3. モジュールをインポート
from jpy_datareader.estat import StatsDataReader

学校種別の学習費
https://www.mext.go.jp/b_menu/toukei/chousa03/gakushuuhi/sonota/1399388_00001.htm
https://www.e-stat.go.jp/stat-search/files?page=1&layout=datalist&toukei=00400201&tstat=000001012023&cycle=0&tclass1=000001224200&tclass2=000001224201&tclass3=000001224327&tclass4val=0

https://www.e-stat.go.jp/stat-search/file-download?statInfId=000040233660&fileKind=0
https://www.e-stat.go.jp/stat-search/file-download?statInfId=000040233661&fileKind=0
https://www.e-stat.go.jp/stat-search/file-download?statInfId=000040233662&fileKind=0
https://www.e-stat.go.jp/stat-search/file-download?statInfId=000040233663&fileKind=0

level1,level2,level3,level4,一時的
学習費総額,学校教育費,学校教育費,入学金・入園料,1
学習費総額,学校教育費,学校教育費,入学時に納付した施設整備費等,1
学習費総額,学校教育費,学校教育費,入学検定料,1
学習費総額,学校教育費,学校教育費,授業料,
学習費総額,学校教育費,学校教育費,施設整備費等,
学習費総額,学校教育費,学校教育費,修学旅行費,
学習費総額,学校教育費,学校教育費,校外学習費,
学習費総額,学校教育費,学校教育費,学級・児童会・生徒会費,
学習費総額,学校教育費,学校教育費,その他の学校納付金,
学習費総額,学校教育費,学校教育費,PTA会費,
学習費総額,学校教育費,学校教育費,後援会費,
学習費総額,学校教育費,学校教育費,寄附金,
学習費総額,学校教育費,学校教育費,教科書費・教科書以外の図書費,
学習費総額,学校教育費,学校教育費,学用品・実験実習材料費,
学習費総額,学校教育費,学校教育費,教科外活動費,
学習費総額,学校教育費,学校教育費,通学費,
学習費総額,学校教育費,学校教育費,制服,
学習費総額,学校教育費,学校教育費,通学用品費,
学習費総額,学校教育費,学校教育費,その他学校教育費,
学習費総額,学校給食費,学校給食費,学校給食費,
学習費総額,学校外活動費,補助学習費,家庭内学習費,
学習費総額,学校外活動費,補助学習費,通信教育・家庭教師費,
学習費総額,学校外活動費,補助学習費,学習塾費,
学習費総額,学校外活動費,補助学習費,その他補助学習費,
学習費総額,学校外活動費,その他の学校外活動費,体験活動・地域活動,
学習費総額,学校外活動費,その他の学校外活動費,芸術文化活動,
学習費総額,学校外活動費,その他の学校外活動費,スポーツ・レクリエーション活動,
学習費総額,学校外活動費,その他の学校外活動費,国際交流体験活動,
学習費総額,学校外活動費,その他の学校外活動費,教養・その他,


In [ ]:
import re
from io import StringIO
from typing import Dict, Optional, Tuple

import numpy as np
import pandas as pd

In [ ]:
# URL configuration using f-string for efficiency
BASE_URL = "https://www.e-stat.go.jp/stat-search/file-download"

STAT_IDS = {
    'kindergarten': '000040233660',
    'elementary': '000040233661',
    'middle': '000040233662',
    'high': '000040233663'
}

URLS = {
    level: f"{BASE_URL}?statInfId={stat_id}&fileKind=0"
    for level, stat_id in STAT_IDS.items()
}

SKIPROWS = {
    'kindergarten': 10,
    'elementary': 10,
    'middle': 10,
    'high': 9  # Different from others
}

AGE_OR_GRADE = {
    'kindergarten': '年齢別',
    'elementary': '学年別',
    'middle': '学年別',
    'high': '学年別'
}

NA_VALUES = ["…", "－"]

TIDY_COLUMNS = ["学習費区分", "学校", "学年", "学習費"]

class AgeOffsets:
    """Age offset constants for different education levels."""
    KINDERGARTEN = 2
    ELEMENTARY = 5
    MIDDLE = 11
    HIGH = 14

In [ ]:
LEVEL_MAPPING_CSV  = """level1,level2,level3,level4
学習費総額,学校教育費,学校教育費,入学金・入園料
学習費総額,学校教育費,学校教育費,入学時に納付した施設整備費等
学習費総額,学校教育費,学校教育費,入学検定料
学習費総額,学校教育費,学校教育費,授業料
学習費総額,学校教育費,学校教育費,施設整備費等
学習費総額,学校教育費,学校教育費,修学旅行費
学習費総額,学校教育費,学校教育費,校外学習費
学習費総額,学校教育費,学校教育費,学級・児童会・生徒会費
学習費総額,学校教育費,学校教育費,その他の学校納付金
学習費総額,学校教育費,学校教育費,PTA会費
学習費総額,学校教育費,学校教育費,後援会費
学習費総額,学校教育費,学校教育費,寄附金
学習費総額,学校教育費,学校教育費,教科書費・教科書以外の図書費
学習費総額,学校教育費,学校教育費,学用品・実験実習材料費
学習費総額,学校教育費,学校教育費,教科外活動費
学習費総額,学校教育費,学校教育費,通学費
学習費総額,学校教育費,学校教育費,制服
学習費総額,学校教育費,学校教育費,通学用品費
学習費総額,学校教育費,学校教育費,その他
学習費総額,学校給食費,学校給食費,学校給食費
学習費総額,学校外活動費,補助学習費,家庭内学習費
学習費総額,学校外活動費,補助学習費,通信教育・家庭教師費
学習費総額,学校外活動費,補助学習費,学習塾費
学習費総額,学校外活動費,補助学習費,その他
学習費総額,学校外活動費,その他の学校外活動費,体験活動・地域活動
学習費総額,学校外活動費,その他の学校外活動費,芸術文化活動
学習費総額,学校外活動費,その他の学校外活動費,スポーツ・レクリエーション活動
学習費総額,学校外活動費,その他の学校外活動費,国際交流体験活動
学習費総額,学校外活動費,その他の学校外活動費,教養・その他"""

In [ ]:
def load_raw_mapping():
    """
    Load the raw education cost category mapping without any modifications.
    
    Returns
    -------
    pd.DataFrame
        Raw mapping table with original "その他" labels.
        Contains columns: level1, level2, level3, level4 representing
        the hierarchical structure of education cost categories.
    """
    try:
        mapping = pd.read_csv(StringIO(LEVEL_MAPPING_CSV))
        print("Raw mapping table loaded successfully")
        return mapping
        
    except Exception as e:
        print(f"Failed to load raw mapping table: {e}")
        raise
raw_mapping = load_raw_mapping()

In [ ]:
def create_label_mapping(mapping_df):
    """
    Create a mapping dictionary for updating "その他" labels based on context.
    
    Parameters
    ----------
    mapping_df : pd.DataFrame
        The mapping DataFrame containing education cost categories.
        
    Returns
    -------
    Dict[int, Dict[str, str]]
        Dictionary mapping index positions to context information.
        Contains 'level3' and 'level4' for each position.
    """
    return {
        i: {'level3': row['level3'], 'level4': row['level4']}
        for i, row in mapping_df.iterrows()
    }
label_mapping = create_label_mapping(raw_mapping)
label_mapping

In [ ]:
def process_mapping(mapping):
    """
    Process the raw mapping by disambiguating "その他" labels.
    
    Parameters
    ----------
    mapping : pd.DataFrame
        Raw mapping table with original "その他" labels.
        
    Returns
    -------
    pd.DataFrame
        Processed mapping table with disambiguated "その他" labels.
        "その他" labels are updated to include parent category context.
    """
    try:
        mapping_processed = mapping.copy()
        
        # Disambiguate "その他" labels by adding parent category context
        mask1 = (mapping_processed.level3 == "学校教育費") & (mapping_processed.level4 == "その他")
        mask2 = (mapping_processed.level3 == "補助学習費") & (mapping_processed.level4 == "その他")
        
        mapping_processed.loc[mask1, "level4"] = "その他学校教育費"
        mapping_processed.loc[mask2, "level4"] = "その他補助学習費"
        
        print("Mapping table processed successfully")
        return mapping_processed
        
    except Exception as e:
        print(f"Failed to process mapping table: {e}")
        raise
processed_mapping = process_mapping(raw_mapping)
processed_mapping

In [ ]:
def load_education_data():
    """
    Load education cost data for all education levels.
    
    Returns
    -------
    Dict[str, pd.DataFrame]
        Dictionary containing DataFrames for each education level.
        Keys are 'kindergarten', 'elementary', 'middle', 'high'.
        Each DataFrame contains education cost data with multi-level headers.
    """
    data = {}
    
    for level in ['kindergarten', 'elementary', 'middle', 'high']:
        try:
            print(f"Loading {level} data...")
            
            df = pd.read_excel(
                URLS[level],
                skiprows=SKIPROWS[level],
                header=[0, 1, 2],
                index_col=0,
                na_values=NA_VALUES
            )
            
            # Filter columns based on education level
            header_col = AGE_OR_GRADE[level]
            df = df.loc[:, pd.IndexSlice[:, header_col, :]]
            
            data[level] = df
            print(f"Successfully loaded {level} data")
            
        except Exception as e:
            print(f"Failed to load {level} data: {e}")
            raise
    
    return data

education_data = load_education_data()

In [ ]:
education_data['elementary'].index

In [ ]:
def update_index_labels(df, mapping_dict):
    """
    Original implementation using context tracking.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame whose index labels need to be updated.
    mapping_dict : Dict[int, Dict[str, str]]
        Dictionary mapping (not used in this implementation).
        
    Returns
    -------
    pd.DataFrame
        DataFrame with updated index labels using context tracking method.
    """
    df_copy = df.copy()
    
    # Create new index by replacing labels that exist in mapping_dict
    new_index = []
    context_tracker = None  # Track current context for disambiguating "その他"
    
    for label in df_copy.index:
        # Track context for "その他" disambiguation
        if label in ["学校教育費", "補助学習費"]:
            context_tracker = label
        
        # Replace "その他" based on current context
        if label == "その他" and context_tracker:
            new_index.append(f"{label}{context_tracker}")
        else:
            new_index.append(label)
    
    df_copy.index = new_index
    return df_copy
education_data_updated = {}
for level in education_data:
    education_data_updated[level] = update_index_labels(
        education_data[level], 
        label_mapping
    )

In [ ]:
education_data_updated['elementary']

In [ ]:
def convert_to_tidy(data):
    """
    Convert education data to tidy format.
    
    Parameters
    ----------
    data : Dict[str, pd.DataFrame]
        Dictionary of DataFrames for each education level.
        
    Returns
    -------
    pd.DataFrame
        Combined DataFrame in tidy format with columns:
        ['学習費区分', '学校', '学年', '学習費'].
    """
    tidy_dfs = []
    
    for level, df in data.items():
        try:
            # Stack to create tidy format
            tidy_df = (df.stack([0, 2], future_stack=True)
                      .reset_index()
                      .set_axis(TIDY_COLUMNS, axis=1))
            
            tidy_dfs.append(tidy_df)
            print(f"Converted {level} data to tidy format")
            
        except Exception as e:
            print(f"Failed to convert {level} data to tidy format: {e}")
            raise
    
    return pd.concat(tidy_dfs, ignore_index=True)

df_tidy = convert_to_tidy(education_data_updated)
df_tidy

In [ ]:
def split_school_info(df):
    """
    Split school information into public/private and school type.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with '学校' column containing combined school information.
        
    Returns
    -------
    pd.DataFrame
        DataFrame with separated school information:
        - '公立・私立区分': Public or private classification
        - '学校': School type without public/private prefix
    """
    df_copy = df.copy()
    
    # Extract public/private and school type
    split_df = df_copy["学校"].str.extract(r'^(公立|私立)(.+)$')
    split_df.columns = ["公立・私立区分", "学校"]
    
    # Replace original school column with split information
    df_result = df_copy.drop(columns="学校").join(split_df)
    
    print("School information split successfully")
    return df_result
df_split = split_school_info(df_tidy)
df_split

In [ ]:
def extract_grade_info(grade_str):
    """
    Extract age and grade information from grade string.
    
    Parameters
    ----------
    grade_str : str
        String containing grade information (e.g., "3歳", "第1学年").
        
    Returns
    -------
    Tuple[Optional[int], Optional[int]]
        Tuple containing (age, grade). Returns (age, None) for kindergarten
        age strings, (None, grade) for school grade strings, or (None, None)
        if no pattern matches.
    """
    if "歳" in grade_str:
        match = re.search(r'(\d+)歳', grade_str)
        return int(match.group(1)) if match else None, None
    elif "学年" in grade_str:
        match = re.search(r'第(\d+)学年', grade_str)
        return None, int(match.group(1)) if match else None
    return None, None

def calculate_age_vectorized(df):
    """
    Calculate age using vectorized operations.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with '学校' and '学年' columns.
        
    Returns
    -------
    pd.Series
        Series containing calculated ages based on school type and grade.
        Uses predefined age offsets for different education levels.
    """
    conditions = [
        df["学校"].str.contains("幼稚園"),
        df["学校"].str.contains("小学校"),
        df["学校"].str.contains("中学校"),
        df["学校"].str.contains("高等学校")
    ]
    
    choices = [
        df["学年"] + AgeOffsets.KINDERGARTEN,
        df["学年"] + AgeOffsets.ELEMENTARY,
        df["学年"] + AgeOffsets.MIDDLE,
        df["学年"] + AgeOffsets.HIGH
    ]
    
    return np.select(conditions, choices, default=np.nan)
    
def process_grade_and_age(df):
    """
    Process grade and age information.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with original grade strings in '学年' column.
        
    Returns
    -------
    pd.DataFrame
        DataFrame with processed grade and age columns:
        - '学年': Standardized 1-based grade numbers
        - '年齢': Calculated ages based on school type and grade
    """
    df_copy = df.copy()
    
    # Store original grade string
    df_copy["orig_grade"] = df_copy["学年"]
    
    # Extract age and grade information
    grade_info = df_copy["orig_grade"].apply(extract_grade_info)
    df_copy["age_extracted"] = [info[0] for info in grade_info]
    df_copy["grade_extracted"] = [info[1] for info in grade_info]
    
    # Calculate standardized grade (1-based)
    df_copy["学年"] = np.where(
        df_copy["grade_extracted"].notna(),
        df_copy["grade_extracted"],
        df_copy["age_extracted"] - AgeOffsets.KINDERGARTEN
    ).astype(int)
    
    # Calculate age
    df_copy["年齢"] = np.where(
        df_copy["age_extracted"].notna(),
        df_copy["age_extracted"],
        calculate_age_vectorized(df_copy)
    ).astype(int)
    
    # Clean up temporary columns
    df_copy = df_copy.drop(columns=["orig_grade", "age_extracted", "grade_extracted"])
    
    print("Grade and age processing completed")
    return df_copy
    
df_processed = process_grade_and_age(df_split)  
df_processed

In [ ]:
def merge_with_mapping(df, mapping):
    """
    Merge education data with category mapping.
    
    Parameters
    ----------
    df : pd.DataFrame
        Processed education data with '学習費区分' column.
    mapping : pd.DataFrame
        Category mapping table with hierarchical levels.
        
    Returns
    -------
    pd.DataFrame
        Merged DataFrame with category hierarchy columns:
        ['学習費区分1', '学習費区分2', '学習費区分3', '学習費区分4'].
        Rows with unmapped categories are removed.
    """
    df_merged = df.merge(
        mapping[["level1", "level2", "level3", "level4"]],
        left_on="学習費区分",
        right_on="level4",
        how="left"
    )
    
    # Remove rows where all level columns are null
    df_merged = df_merged.dropna(
        subset=['level1', 'level2', 'level3', 'level4'], 
        how='all'
    )
    
    # Remove original category column
    df_merged = df_merged.drop(columns=['学習費区分'])
    
    # Rename level columns
    df_merged = df_merged.rename(columns={
        'level1': '学習費区分1',
        'level2': '学習費区分2',
        'level3': '学習費区分3',
        'level4': '学習費区分4',
    })
    
    print("Data merged with mapping successfully")
    return df_merged

df_merged = merge_with_mapping(df_processed, processed_mapping).fillna(0)
df_final = df_merged[['公立・私立区分', '学校', '学年', '年齢', '学習費区分1', '学習費区分2', '学習費区分3', '学習費区分4', '学習費']].sort_values(['公立・私立区分', '年齢'])
df_final

In [ ]:
df_final.to_csv("tuition.csv", index=False)

In [ ]:
# 学習費区分1～4 を行の MultiIndex、指定の列をカラムに、学習費を値にするピボット
df_table = df_merged.pivot(
    index=['公立・私立区分', '学校', '学年', '年齢'],
    columns=['学習費区分1', '学習費区分2', '学習費区分3', '学習費区分4'],
    values='学習費'
)

# 必要に応じて、列の並び替えやソートを行うこともできます
df_table = df_table.sort_index(level=[0, 3], axis=0).sort_index(axis=1)

# 確認
df_table

| 調査名                         | 実施機関       | データの種類             | 年齢別情報 | 備考                                                             |
|------------------------------|----------------|--------------------------|------------|------------------------------------------------------------------|
| 賃金構造基本統計調査             | 厚生労働省       | 賃金（所定内・外、賞与など） | ◎ あり      | 最も詳細な個人ベースの賃金統計。年齢×学歴×勤続年数×職種など多変量分析可。     |
| 民間給与実態統計調査             | 国税庁          | 年間給与（所得）          | ◎ あり      | 全国の給与所得者の年収実態。平均・中央値・分布あり。正規/非正規別も可。         |
| 毎月勤労統計調査                 | 厚生労働省       | 月次の給与・労働時間       | ◯ 一部あり   | 年報などで年齢別給与も公表。短期の変化や景気動向把握に向く。                  |
| 家計調査                       | 総務省統計局     | 世帯の実収入・支出        | ◯ あり      | 勤労者世帯の世帯主年齢別の平均月収がわかる。世帯単位のデータ。                |
| 全国家計構造調査（旧・全国消費実態調査） | 総務省統計局     | 所得・支出・貯蓄・負債等    | ◯ あり      | 5年ごと実施。世帯主年齢別に詳細な所得内訳がわかる。                           |
| 国民生活基礎調査                 | 厚生労働省       | 世帯の所得・生活実態       | ◯ あり      | 世帯主年齢別の所得平均・中央値・分布あり。福祉・医療と併せた分析が可能。         |
| 就業構造基本調査                 | 総務省統計局     | 職業・就業形態・月収階級等  | △（金額なし、階級）| 月収階級別の年齢分布あり。職種・雇用形態とのクロス集計が可能。                  |
| 労働力調査                     | 総務省統計局     | 雇用状況（就業・失業）     | ◯（人数のみ）| 年齢別の就業者数・非労働力人口は把握できるが、賃金金額はなし。                  |



国民生活基礎調査
１世帯当たり平均所得金額－世帯人員１人当たり平均所得金額，世帯主の年齢（10歳階級）・年次別　
0004021242
平均所得金額－平均世帯人員－平均有業人員，世帯主の年齢（10歳階級）別
0004021274

家計調査
二人以上の世帯
用途分類（世帯主の年齢階級別）
0002070010
用途分類（世帯主の年齢階級別）　年次
0002070011

単身世帯
用途分類（年齢階級別）
0003000795
0002190004

全国家計構造調査
世帯の種類(3区分)，世帯区分(4区分)，世帯主の性別(3区分)，世帯主の年齢階級(32区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426498
世帯の種類(3区分)，世帯主の勤め先企業規模(20区分)，世帯主の年齢階級(32区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426474
世帯人員(8区分)，有業人員(7区分)，世帯主の年齢階級(32区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426475
世帯の種類(3区分)，世帯主の年齢階級(32区分)，世帯主の職業(17区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426476
世帯の種類(3区分)，世帯主の年齢階級(32区分)，公的年金・恩給受給額階級(14区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426478
世帯の種類(3区分)，世帯主の年齢階級(32区分)，年間収入階級(44区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426483
世帯の種類(3区分)，世帯主の年齢階級(32区分)，年間収入十分位階級(13区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426484
世帯の種類(3区分)，世帯主の年齢階級(32区分)，年間収入五分位階級(6区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426485
世帯主の年齢階級(32区分)，有業人員(7区分)，高齢者世帯類型(21区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426489
世帯主の年齢階級(32区分)，世帯主の従業上の地位(11区分)，世帯主の配偶者の有無(30区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426490
世帯主の年齢階級(32区分)，未婚の子供の数(4区分)，世帯主の配偶者の有無(30区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426495
世帯主の年齢階級(32区分)，有業人員(7区分)，非就業者の有無(40区分)，所得構成(44区分)別１世帯当たり年間収入額－全国
0003426496


民間給与
全国計表　第10表　事業所規模別及び年齢階層別の給与所得者数・給与額　（1年を通じて勤務した給与所得者）　（2014年～）
0004009621
全国計表　第11表　企業規模別及び年齢階層別の給与所得者数・給与額　（1年を通じて勤務した給与所得者）　（2014年～）
0004009622
全国計表　第12表　業種別及び年齢階層別の給与所得者数・給与額　（1年を通じて勤務した給与所得者）　（2014年～）
0004009623
全国計表　第12表　業種別及び年齢階層別の給与所得者数・給与額　（1年を通じて勤務した給与所得者（乙欄適用者を除く））（2014年～）
0004009625

賃金構造基本統計調査
https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00450091&tstat=000001011429&cycle=0&tclass1val=0&metadata=1&data=1
一般_産業大・中分類_年齢階級別DB
0003425893
一般_役職_年齢階級別DB
0003426253
一般_都道府県別_年齢階級別DB
0003426933

